In [80]:
import json
import pandas as pd
import requests

In [81]:
df = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

In [91]:
mrt = pd.read_excel('mrt.xlsx')
mrt

OBJECTID                     STN_NAME STN_NO           X           Y  \
0          12        ADMIRALTY MRT STATION   NS10  24402.1063  46918.1131   
1          16         ALJUNIED MRT STATION    EW9  33518.6049  33190.0020   
2          33       ANG MO KIO MRT STATION   NS16  29807.2655  39105.7720   
3          81            BAKAU LRT STATION    SE3  36026.0821  41113.8766   
4          80          BANGKIT LRT STATION    BP9  21248.2460  40220.9693   
..        ...                          ...    ...         ...         ...   
182       175  WOODLANDS SOUTH MRT STATION    TE3  23607.8309  45444.7113   
183       146        WOODLEIGH MRT STATION   NE11  32173.3186  35706.3794   
184         6          YEW TEE MRT STATION    NS5  18438.9791  42158.0124   
185        41     YIO CHU KANG MRT STATION   NS15  29294.1283  40413.0820   
186        13           YISHUN MRT STATION   NS13  28187.6787  45686.0701   

     Latitude   Longitude   COLOR  
0    1.440585  103.800998     RED  
1    1.316433  103.882893   GREEN  
2    1.369933  103.849553     RED  
3    1.388093  103.905418  OTHERS  
4    1.380018  103.772667  OTHERS  
..        ...         ...     ...  
182  1.427260  103.793863  OTHERS  
183  1.339190  103.870808  PURPLE  
184  1.397535  103.747431     RED  
185  1.381756  103.844944     RED  
186  1.429443  103.835004     RED  

[187 rows x 8 columns]

In [86]:
df = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
df['month']=pd.to_datetime(df['month'])
df['Year'] = df['month'].apply(lambda x: x.year)
df['Month'] = df['month'].apply(lambda y: y.month)
df = df.drop(columns = ['month','Month'])
df = df[df['Year'] == 2019].iloc[0:100]
df2 = df[df['Year'] == 2019].iloc[0:100]
df.head()

town flat_type block       street_name storey_range  \
42070  ANG MO KIO    3 ROOM   330  ANG MO KIO AVE 1     01 TO 03   
42071  ANG MO KIO    3 ROOM   215  ANG MO KIO AVE 1     04 TO 06   
42072  ANG MO KIO    3 ROOM   225  ANG MO KIO AVE 1     07 TO 09   
42073  ANG MO KIO    3 ROOM   225  ANG MO KIO AVE 1     01 TO 03   
42074  ANG MO KIO    3 ROOM   333  ANG MO KIO AVE 1     01 TO 03   

       floor_area_sqm      flat_model  lease_commence_date  \
42070            68.0  New Generation                 1981   
42071            73.0  New Generation                 1976   
42072            67.0  New Generation                 1978   
42073            67.0  New Generation                 1978   
42074            68.0  New Generation                 1981   

          remaining_lease  resale_price  Year  
42070   61 years 01 month      270000.0  2019  
42071  56 years 04 months      295000.0  2019  
42072   58 years 01 month      270000.0  2019  
42073            58 years      230000.0  2019  
42074            61 years      262500.0  2019

In [87]:
df2['address'] =  df['block'] + " " + df['street_name']
df2.head()

town flat_type block       street_name storey_range  \
42070  ANG MO KIO    3 ROOM   330  ANG MO KIO AVE 1     01 TO 03   
42071  ANG MO KIO    3 ROOM   215  ANG MO KIO AVE 1     04 TO 06   
42072  ANG MO KIO    3 ROOM   225  ANG MO KIO AVE 1     07 TO 09   
42073  ANG MO KIO    3 ROOM   225  ANG MO KIO AVE 1     01 TO 03   
42074  ANG MO KIO    3 ROOM   333  ANG MO KIO AVE 1     01 TO 03   

       floor_area_sqm      flat_model  lease_commence_date  \
42070            68.0  New Generation                 1981   
42071            73.0  New Generation                 1976   
42072            67.0  New Generation                 1978   
42073            67.0  New Generation                 1978   
42074            68.0  New Generation                 1981   

          remaining_lease  resale_price  Year               address  
42070   61 years 01 month      270000.0  2019  330 ANG MO KIO AVE 1  
42071  56 years 04 months      295000.0  2019  215 ANG MO KIO AVE 1  
42072   58 years 01 month      270000.0  2019  225 ANG MO KIO AVE 1  
42073            58 years      230000.0  2019  225 ANG MO KIO AVE 1  
42074            61 years      262500.0  2019  333 ANG MO KIO AVE 1

In [88]:
# This cell ran for >30 min to retrieve 8k+ records

latitude = []
longitude = []
postal_code = []
full_address = []
missing = []

for row in df2['address']:
    query_address = row
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal=' + str(query_address) + '&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    # Convert JSON into Python Object
    data_geo_location = json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        full_address.append(data_geo_location['results'][0]['ADDRESS'])
    else:
        missing.append(query_address)

In [89]:
df2['latitude'] = latitude
df2['longitude'] = longitude
df2['full_address'] = full_address
df2['postal_code'] = postal_code

In [90]:
df2.head()

town flat_type block       street_name storey_range  \
42070  ANG MO KIO    3 ROOM   330  ANG MO KIO AVE 1     01 TO 03   
42071  ANG MO KIO    3 ROOM   215  ANG MO KIO AVE 1     04 TO 06   
42072  ANG MO KIO    3 ROOM   225  ANG MO KIO AVE 1     07 TO 09   
42073  ANG MO KIO    3 ROOM   225  ANG MO KIO AVE 1     01 TO 03   
42074  ANG MO KIO    3 ROOM   333  ANG MO KIO AVE 1     01 TO 03   

       floor_area_sqm      flat_model  lease_commence_date  \
42070            68.0  New Generation                 1981   
42071            73.0  New Generation                 1976   
42072            67.0  New Generation                 1978   
42073            67.0  New Generation                 1978   
42074            68.0  New Generation                 1981   

          remaining_lease  resale_price  Year               address  \
42070   61 years 01 month      270000.0  2019  330 ANG MO KIO AVE 1   
42071  56 years 04 months      295000.0  2019  215 ANG MO KIO AVE 1   
42072   58 years 01 month      270000.0  2019  225 ANG MO KIO AVE 1   
42073            58 years      230000.0  2019  225 ANG MO KIO AVE 1   
42074            61 years      262500.0  2019  333 ANG MO KIO AVE 1   

                 latitude           longitude  \
42070  1.3624318640000002  103.85103070000001   
42071  1.3665583019999998  103.84162409999999   
42072  1.3673961280000002           103.83815   
42073  1.3673961280000002           103.83815   
42074  1.3613425559999999  103.85169859999999   

                                            full_address postal_code  
42070  330 ANG MO KIO AVENUE 1 TECK GHEE VIEW SINGAPO...      560330  
42071  215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...      560215  
42072  225 ANG MO KIO AVENUE 1 HDB-ANG MO KIO SINGAPO...      560225  
42073  225 ANG MO KIO AVENUE 1 HDB-ANG MO KIO SINGAPO...      560225  
42074  333 ANG MO KIO AVENUE 1 TECK GHEE VIEW SINGAPO...      560333

In [105]:
# get the nearest mrt
from geopy.distance import geodesic

def find_nearest(add_lat_lon_start,add_lat_lon_end):
    
    results = {}
    # first column must be address
    for index,flat in enumerate(add_lat_lon_start.iloc[:,14]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (add_lat_lon_start.iloc[index,12],add_lat_lon_start.iloc[index,13])
        nearest_stn = ['',10]
        for ind, stn in enumerate(add_lat_lon_end.iloc[:,1]):
            stn_loc = (add_lat_lon_end.iloc[ind,5],add_lat_lon_end.iloc[ind,6])
            distance = geodesic(flat_loc,stn_loc)

            if distance < nearest_stn[1]:
                nearest_stn[0] = stn
                nearest_stn[1] = distance

        results[flat] = nearest_stn
        
    return results

a = find_nearest(df2,mrt)
a['330 ANG MO KIO AVENUE 1 TECK GHEE VIEW SINGAPORE 560330']

['ANG MO KIO MRT STATION', Distance(0.8455910068343545)]